In [159]:
is_stack = [True, False][0]
debug = False
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

#========================================================================
# Keras 
# Corporación Favorita Grocery Sales Forecasting
from sklearn.linear_model import Ridge
#========================================================================


#========================================================================
# Args
out_part = ['', 'part', 'all'][0]
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
stack_name='ridge'
submit = pd.read_csv('../input/sample_submission.csv')
model_type='ridge'
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
seed = 328
#========================================================================


#========================================================================
# Data Load 
print("Preparing dataset...")

if is_stack:
    is_clf_out = [True, False][1]
    is_no_out_flg = [True, False][1]
    is_rm_out = [True, False][1]
    #========================================================================
    # Base Model Load
    base = utils.read_df_pkl('../input/base_no_out_clf.gz').set_index(key)
    ens_list = glob.glob('../ensemble/*.gz')
    # ens_list = ['../ensemble/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz']
    for path in ens_list:
        try:
            cv = re.search(r'CV([^/.]*)_LB.gz', path).group(1)
        except AttributeError:
            cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
        
        try:
            blend = utils.read_df_pkl(path).set_index(key)['pred_mean']
        except KeyError:
            blend = utils.read_df_pkl(path).set_index(key)['prediction']
        if path.count('lgb'):
            base[f'base_lgb_{cv}'] = blend
        elif path.count('NN'):
            base[f'base_NN_{cv}'] = blend
    #========================================================================
    
    #========================================================================
    # Clf Out Model
    if is_clf_out:
        out_list = glob.glob('../clf_min_thres_ensemble/*.gz')
        out_list = ['../clf_min_thres_ensemble/0212_005_clf_out_lgb_out_partclf_out_row167982_lr0.01_235feats_10seed_57leaves_iter1062_OUT0_CV2-1684031012798815_LB.gz', '../clf_min_thres_ensemble/0211_144_elo_NN_stack_E1_row167982_235feat_const2_lr0.001_batch128_epoch30_CV2.1341374191726357.gz']
        for path in out_list:
            cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
            try:
                blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
            except KeyError:
                blend = utils.read_pkl_gzip(path).set_index(key)['prediction']
            base[f"clf_out_{cv}"] = blend
        out_cols = [col for col in base.columns if col.count('clf_out_2')]
    #========================================================================
    
    #========================================================================
    # No Out Flg Model
    if is_no_out_flg:
        no_out_flg_list = glob.glob('../no_out_flg_ensemble/*.gz')
        no_out_flg_list = ['../no_out_flg_ensemble/0212_010_no_out_flg_lgb_out_partno_out_flg_row40858_lr0.01_235feats_10seed_57leaves_iter784_OUT0_CV1-4795858837810465_LB.gz']
    
        for path in no_out_flg_list:
            cv = re.search(r'CV([^/.]*).gz', path.replace('.', '-')).group(1)
            try:
                blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
            except KeyError:
                blend = utils.read_pkl_gzip(path).set_index(key)['prediction']
            base[f"no_out_flg_{cv}"] = blend
        no_out_cols = [col for col in base.columns if col.count('no_out_flg_1')]
    #========================================================================
    
    #========================================================================
    # Classifier
    clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
    clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
    clf['pred_mean_2'] = clf_2['pred_mean']
    clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
    # clf['clf_pred'] =  clf['pred_mean']
    #========================================================================
    
    #========================================================================
    # Indexをそろえる
    base['clf_pred'] = clf['clf_pred']
#     base_cols = [col for col in base.columns if col.count('base_')]
    base_lgb_cols = [col for col in base.columns if col.count('base_lgb')]
    base_NN_cols = [col for col in base.columns if col.count('base_NN')]
    # base = base[[target, 'clf_pred', 'no_out_flg'] + base_cols + out_cols + no_out_cols]
    df_stack = base[[target, 'no_out_flg'] + base_lgb_cols + base_NN_cols]
    #========================================================================
    
else:
    win_path = f'../features/4_winner/*.gz'
    # Ensemble 1
    win_path = f'../model/LB3670_70leaves_colsam0322/*.gz'
    # Ensemble 2
    # win_path = f'../model/E2_lift_set/*.gz'
    # Ensemble 3
    # win_path = f'../model/E3_PCA_set/*.gz'
    
    win_path_list = glob.glob(win_path)
    
    base = utils.read_df_pkl('../input/base_term*0*')[[key, target, 'first_active_month']]
    base_train = base[~base[target].isnull()].reset_index(drop=True)
    base_test = base[base[target].isnull()].reset_index(drop=True)
    feature_list = utils.parallel_load_data(path_list=win_path_list)
    df = pd.concat(feature_list, axis=1)
    train = pd.concat([base_train, df.iloc[:len(base_train), :]], axis=1)
    test = pd.concat([base_test, df.iloc[len(base_train):, :].reset_index(drop=True)], axis=1)
    
    train.reset_index(inplace=True, drop=True)
    test.reset_index(inplace=True , drop=True)

if debug:
    train = train.head(10000)
    test = test.head(2000)
#========================================================================

#========================================================================
# 正規化の前処理(Null埋め, inf, -infの処理) 
for col in train.columns:
    if col in ignore_list: continue
        
    train[col] = impute_feature(train, col)
    test[col] = impute_feature(test, col)
#========================================================================

#========================================================================
# Cleansing Check
def clean_check(df, col):
#     if col in ignore_list: continue
#     train[col] = impute_feature(train, col)
#     test[col] = impute_feature(test, col)
    length = len(df)
    tmp = df[col].dropna().shape[0]
    if length - tmp>0:
        print(f"Null is {length-tmp}")
        
    inf_max = df[col].max()
    inf_min = df[col].min()
    inf_max_2 = df[col].sort_values().values[-1]
    inf_min_2 = df[col].sort_values().values[0]
    if inf_max==np.inf or inf_min==-np.inf:
        print(1, col, inf_max, inf_min)
    if inf_max_2==np.inf or inf_min_2==-np.inf:
        print(2, col, inf_max, inf_min)
#========================================================================

  0%|          | 0/1 [00:00<?, ?it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Preparing dataset...


100%|██████████| 1/1 [00:00<00:00,  7.73it/s]


In [160]:
base_lgb_cols = sorted(base_lgb_cols)
base_NN_cols = sorted(base_NN_cols)
stack_cols = base_lgb_cols + base_NN_cols
# stack_cols = base_lgb_cols
if key in base.columns:
    df_stack = base[[key, target, 'no_out_flg'] + stack_cols]
else:
    df_stack = base[[target, 'no_out_flg'] + stack_cols]

if key in df_stack.columns:
    train = df_stack[~df_stack[target].isnull()]
    test = df_stack[df_stack[target].isnull()]
else:
    train = df_stack[~df_stack[target].isnull()].reset_index()
    test = df_stack[df_stack[target].isnull()].reset_index()
train.head()

,card_id,target,no_out_flg,base_lgb_3-6206463759490277,base_lgb_3-63947267133467,base_lgb_3-64344499327249,base_lgb_3-6439834621816565,base_NN_3-766855322520765,base_NN_3-7790332090708425
0,C_ID_92a2005557,-0.820283,1.0,-0.346904,-0.298933,-0.208776,-0.355493,-0.210441,-0.276462
1,C_ID_3d0044924f,0.392913,0.0,-0.568272,-0.405721,-0.318021,-0.485598,0.808078,0.175963
2,C_ID_d639edf6cd,0.688056,0.0,0.683088,0.568874,0.524642,0.592692,0.895201,0.644744
3,C_ID_186d6a6901,0.142495,0.0,0.173934,0.159495,0.168799,0.016254,0.482479,0.332322
4,C_ID_cdbd2c0db2,-0.159749,1.0,-0.109142,-0.302391,-0.029702,-0.133293,-0.339484,-0.057478


In [161]:
#========================================================================
# CVの準備
fold = 6
# kfold = ods_kfold(train=train, seed=328, fold=fold)
# train.drop('rounded_target', axis=1, inplace=True)
kfold = utils.read_pkl_gzip('../input/ods_kfold.gz')
submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
model_list = []
result_list = []
score_list = []
val_pred_list = []
test_pred = np.zeros(len(test))

ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
# ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']
use_cols = [col for col in train.columns if col not in ignore_list]
scaler = StandardScaler()
scaler.fit(pd.concat([train[use_cols], test[use_cols]]))
x_test = scaler.transform(test[use_cols])

Y = train[target]
y_mean = Y.mean()
#========================================================================
    
print(f"Train: {train.shape} | Test: {test.shape}")
    
#========================================================================
# NN Model Setting 
fit_intercept = True
alpha = 0.4
max_iter = 1000
normalize = False
tol = 0.01
param_list = np.arange(1, 10, 1)
param_list = [0]
# param_list = np.arange(0.1, 1.0, 0.1)

for param in param_list:

    result_list = []
    ridge = Ridge(solver='auto', fit_intercept=fit_intercept, alpha=alpha, max_iter=max_iter, normalize=normalize, tol=tol)
#     ridge = Ridge(solver='auto', fit_intercept=True, alpha=0.4, max_iter=200, normalize=False, tol=0.01)
    #========================================================================
    
    #========================================================================
    # Train & Prediction Start
    for fold_no, (trn_idx, val_idx) in enumerate(zip(*kfold)):
        if key not in train.columns:
            train = train[~train[target].isnull()].reset_index()
            test = test[test[target].isnull()].reset_index() 
    
        #========================================================================
        # Make Dataset
        X_train, y_train = train.loc[train[key].isin(trn_idx), :][use_cols], Y.loc[train[key].isin(trn_idx)]
        X_val, y_val = train.loc[train[key].isin(val_idx), :][use_cols], Y.loc[train[key].isin(val_idx)]
        
        X_train[:] = scaler.transform(X_train)
        X_val[:] = scaler.transform(X_val)
        X_train = X_train.as_matrix()
        X_val = X_val.as_matrix()
        #========================================================================
        
        # Fitting
        ridge.fit(X_train, y_train)
        
        # Prediction
        y_pred = ridge.predict(X_val)
        test_pred += ridge.predict(x_test)
        
        # Stack Prediction
    #     df_pred = train.iloc[val_idx, :][[key, target]].copy()
        df_pred = train.loc[train[key].isin(val_idx), :][[key, target]].copy()
        df_pred['prediction'] = y_pred
        result_list.append(df_pred)
        
        # Scoring
        err = (y_val - y_pred)
        score = np.sqrt(mean_squared_error(y_val, y_pred))
        print(f'RMSE: {score} | SUM ERROR: {err.sum()}')
        score_list.append(score)
        #========================================================================
    
    cv_score = np.mean(score_list)
    
    #========================================================================
    # Stacking
    test_pred /= fold_no+1
    test['prediction'] = test_pred
    stack_test = test[[key, 'prediction']]
    result_list.append(stack_test)
    df_pred = pd.concat(result_list, axis=0, ignore_index=True).drop(target, axis=1)
    if key not in base:
        base.reset_index(inplace=True)
    df_pred = base[[key, target]].merge(df_pred, how='inner', on=key)
    print(f"Stacking Shape: {df_pred.shape}")
    #========================================================================
    
    #========================================================================
    # outlierに対するスコアを出す
    if key not in train.columns:
        train.reset_index(inplace=True)
    out_ids = train.loc[train.target<-30, key].values
    out_val = train.loc[train.target<-30, target].values
    out_pred = df_pred[df_pred[key].isin(out_ids)]['prediction'].values
    out_score = np.sqrt(mean_squared_error(out_val, out_pred))
    #========================================================================
    
    print(f'''
    #========================================================================
    # CV SCORE AVG: {cv_score}
    # OUT SCORE: {out_score}
    #========================================================================''')
    
#========================================================================
# Submission
df_pred.set_index(key, inplace=True)
submit[target] = df_pred['prediction']
submit_path = f'../submit/{start_time[4:12]}_submit_RIDGE_STACKING_{model_type}_{len(use_cols)}models_OUT{str(out_score)[:7]}_CV{cv_score}_LB.csv'
submit.to_csv(submit_path, index=True)
display(submit.head())
#========================================================================

# Save Stack
# utils.to_pkl_gzip(path=f"../stack/{start_time[4:12]}_stack_{model_type}_alpha{alpha}_{len(use_cols)}feats_tol{tol}_iter{max_iter}_OUT{str(out_score)[:7]}_CV{str(cv_score).replace('.', '-')}_LB" , obj=df_pred)
#========================================================================

Train: (201917, 9) | Test: (123623, 9)
RMSE: 3.5417760271899903 | SUM ERROR: 1233.645569206881
RMSE: 3.6520669388353353 | SUM ERROR: 151.69401459318834
RMSE: 3.645319536331285 | SUM ERROR: -1050.706610428527
RMSE: 3.5946274509871237 | SUM ERROR: -1444.359444298121
RMSE: 3.600140492619795 | SUM ERROR: 1842.1538672907138
RMSE: 3.628358406389664 | SUM ERROR: -629.295183034454
Stacking Shape: (325540, 3)

    #========================================================================
    # CV SCORE AVG: 3.6103814753921992
    # OUT SCORE: 29.93321181151248
    #========================================================================


,target
card_id,
C_ID_0ab67a22ab,-1.645973
C_ID_130fd0cbdd,-0.655524
C_ID_b709037bc5,-1.969370
C_ID_d27d835a9f,-0.222077
C_ID_2b5e3df5c2,-1.431340
